**Following guidelines set expectations for participant behaviour during workshop activities. They also ensure that the class environment is welcoming, inclusive, and respectful.**
- Where a discussion is taking place, allow everyone a chance to speak
- Listen respectfully, without interrupting and with an open mind to understanding others’ views
- Be professional and productive, and always share your ideas, your opinion matters, we all can learn something from each other
- Personal information that comes up in the conversation should be kept confidential
- Avoid inflammatory language
- Avoid assumptions about any member of the class or generalisations about social groups

**You have now joined a group of fellow analysts in a workshop:**
#### Outcomes:

- To be able to answer the questions related SQL queries and Joins

#### Note:
- We understand learners will progress at their own speed
- Tutors will be on hand to answer questions

## ☑️ Combining multiple tables

- Complete the following questions
- Make sure you run the following code cells before you attempt any of the questions
- We will work with `flights.db` database through this workbook

First, import pandas and sqlite3 libraries and create the connection to the `flights.db` database, located in the `data` folder:

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/flights.db')

A database might have multiple tables. It's a good idea to do an initial exploration of the database by first querying the `sqlite_master` table to see what tables are in the database.

Run the following code cell to show all the tables in the `flights.db` database:

In [ ]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type = 'table';
"""
df = pd.read_sql_query(query, conn)
df


Run the following code cell to show the schema for each table in `flights.db` database:

In [ ]:
for table in ['airports','airlines','routes']:
    
    query = f"""
    SELECT sql 
    FROM sqlite_master 
    WHERE name = '{table}';
    """
    
    df = pd.read_sql_query(query, conn)
    print(''.join(df.values[0, 0]))

Run the following code cells to show the contents for each table in `flights.db` database: `['airports','airlines','routes']`

In [ ]:
query = """
SELECT *
FROM airports;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

In [ ]:
query = """
SELECT *
FROM airlines;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

In [ ]:
query = """
SELECT *
FROM routes;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

**Q1)** Combine `routes` and `airlines` tables using a `INNER JOIN`.

- Please note `routes.airline_id` column and `airlines.id` column are related in the two tables: `routes.airline_id=airlines.id`

- Show only `routes.airline`, `routes.airline_id`, `airlines.name` columns in the final result

See below code syntax for some guidance:
```SQL
SELECT <column_names>
FROM <table1>
INNER JOIN <table2>
ON table1.column_name = table2.column_name;
```
Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [ ]:
query = """
SELECT routes.airline, routes.airline_id, airlines.name
FROM routes 
INNER JOIN airlines 
ON routes.airline_id = airlines.id;
""" 
df = pd.read_sql_query(query, conn)
df

**Q2)** Now combine `routes` and `airlines` tables using a `LEFT JOIN`

- Please note `routes.airline_id` column and `airlines.id` column are related in the two tables: `routes.airline_id=airlines.id`

- Show only `routes.airline`, `routes.airline_id`, `airlines.name` columns

See below code syntax for some guidance:
```SQL
SELECT <column_names>
FROM <table1>
LEFT JOIN <table2>
ON table1.column_name = table2.column_name;
```
Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [ ]:
query = """
SELECT routes.airline, routes.airline_id, airlines.name
FROM routes
LEFT JOIN airlines
ON routes.airline_id = airlines.id;
"""
df = pd.read_sql_query(query, conn)
df

It is advisable to take a moment to reflect upon the outcomes of questions 1 and 2, ensuring a thorough understanding of the results. Additionally, make sure to jot down any intriguing observations that arise from the analysis.

In [ ]:
#add your notes below

#INNER JOIN indeed produces a result with fewer or equal rows compared to the original tables,
#whereas LEFT JOIN can potentially result in a larger number of rows in the output compared to the left table.


**Q3)** Get the `latitude` and `longitude` for the *origin* of each route, along with the name of the `city` where the origin airport is located.

We suggest dealing with this in incremental steps:
- Use a `INNER JOIN` to combine the `routes` and `airports` tables

- Note `routes.source_id` column and `airports.id` column should be connected in the two tables: `routes.source_id=airports.id`

- Show only `airports.city`, `airports.longitude`, `airports.latitude` columns

- Limit the results to 5 records only

See below code syntax for some guidance:
```SQL
SELECT <column_names>
FROM <table1>
INNER JOIN <table2>
ON table1.column_name = table2.column_name
LIMIT 5;
```

In [ ]:
#add your code below
#query = ...

query = """
SELECT airports.city, airports.longitude, airports.latitude
FROM routes
INNER JOIN airports
ON routes.source_id = airports.id
LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

**Q4)** We can also get the destination coordinates (latitude and longitude) by matching *destination* id to the airport id. Add an additional `INNER JOIN` to the previous query, and select the corresponding columns to return the destination coordinates.

Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [ ]:
query = """
SELECT 
     sa.city AS origin_city
    ,sa.longitude AS origin_lon
    ,sa.latitude AS origin_lat
    ,da.city AS destination_city
    ,da.longitude AS destination_lon
    ,da.latitude AS destination_lat
FROM routes

INNER JOIN airports AS sa
ON sa.id = routes.source_id

INNER JOIN airports AS da
ON da.id = routes.dest_id

LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

The two INNER JOIN statements in the SQL query link the routes table with the airports table, 
enabling the retrieval of detailed information about the origin and destination airports for each flight route.

The first join connects the routes table to the airports table for the origin airport, 
using the alias **sa** for the **source airport**. 
It matches the id of the airports table with the source_id in the routes table, thereby adding the city, longitude, and latitude of the origin airport to the result set. 

The second join serves a similar purpose for the **destination airport**, using the alias **da**. 
It matches the id of the airports table with the dest_id in the routes table, incorporating the city, longitude, and latitude of the destination airport into the results. 

Together, these two joins enrich the output by providing a comprehensive view of the routes, offering valuable geographical information about where flights originate and where they are headed.

**Q5)** Find the number of routes per airline. Optionally see if you can sort the results in descending order. 

We suggest dealing with this in incremental steps:
- Use a `INNER JOIN` to combine the `routes` and `airlines` tables

- Note `routes.airline_id` column and `airlines.id` column should be connected in the two tables: `routes.airline_id=airlines.id`

- Remember to `GROUP BY` using `routes.airline_id` column

- Find the number of routes per airline, consider using `COUNT(*)` function

- Show only `airline_id`, `airlines.name` columns along with the `COUNT` of routes per each airline

- Use `ORDER BY` to sort the results in descending order

- Limit the results to 5 records only



See below code syntax for some guidance:
```SQL
SELECT <column_names>, COUNT(*) 
FROM <table1>
INNER JOIN <table2>
ON table1.column_name = table2.column_name
GROUP BY <column_name>
ORDER BY <column_name> DESC
LIMIT 5;
```

In [ ]:
#add your code below
#query = ...

query = """
SELECT routes.airline_id, airlines.name, COUNT(*) AS nroutes
FROM routes
INNER JOIN airlines
ON routes.airline_id=airlines.id
GROUP BY routes.airline_id
ORDER BY nroutes DESC
LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

## ☑️ SQL Window Functions

- Complete the following questions
- Make sure you run the following code cells before you attempt any of the questions
- We will work with `flights.db` database through this workbook

First, import pandas and sqlite3 libraries and create the connection to the `flights.db` database, located in the `data` folder:

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/flights.db')

Run the following code cell to show all the tables in the `flights.db` database:

The flights database contains three tables:  `airports, airlines, and routes`:

In [ ]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type = 'table';
"""
df = pd.read_sql_query(query, conn)
df


**Q6)** Refer to the `airports` table. For each `timezone`, **rank** each airport by their `latitude` (ascending).

We suggest dealing with this in incremental steps:

- Show only `name`, `city`, `country` columns from the `airports` table

- Remember to use `RANK()` function alongside `OVER()` keyword with the arguments: `PARTITION BY` and `ORDER BY`

- Note the rows should be partition by `timezone` and sorted in ascending order of `latitude`

See below code syntax for some guidance:
```SQL
SELECT <column_names>
RANK() OVER(PARTITION BY <column_name> ORDER BY <column_name> ASC)
FROM airports;
```

In [ ]:
#add your code below
#query = ...

query = """
SELECT name, city, country, 
RANK() OVER (PARTITION BY timezone ORDER BY latitude ASC)
FROM airports;
"""
df = pd.read_sql_query(query, conn)
df

**Q7)** For each airline, get the number of flights that airline runs that leave per country.

For example, if "Adria Airways" have flights from two different cities in "Albania", there should be one row "Adria Airways, Albania, 2".

Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [ ]:
query = """
SELECT DISTINCT al.name, ap.country,
COUNT(*) OVER(PARTITION BY al.name, ap.country ORDER BY al.name ) AS num_flights
FROM routes
INNER JOIN airports AS ap ON ap.id = routes.source_id
INNER JOIN airlines AS al ON al.id = routes.airline_id
LIMIT 50
"""
df = pd.read_sql_query(query, conn)
df.head(10)

The code retrieves distinct airline names and their corresponding countries from the routes, airports, and airlines tables (all three tables).

It uses INNER JOIN statements to link the three tables based on the source airport and airline IDs. 

The query employs the DISTINCT keyword to ensure that only unique combinations of airline names and countries are returned, along with a window function to count the number of flights for each unique **airline-country combination**.